In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install faker
!pip install nest-asyncio
!apt-get update # to update ubuntu to correctly run apt install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InReleas

In [2]:
!pip install playwright
!playwright install


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 43.1 MB/s eta 0:00:00
145.2 Mb [] 0% 0.0s145.2 Mb [] 0% 23.5s145.2 Mb [] 0% 14.9s145.2 Mb [] 0% 8.4s145.2 Mb [] 1% 5.7s145.2 Mb [] 2% 4.0s145.2 Mb [] 3% 3.3s145.2 Mb [] 3% 3.5s145.2 Mb [] 4% 3.2s145.2 Mb [] 5% 2.9s145.2 Mb [] 7% 2.4s145.2 Mb [] 8% 2.2s145.2 Mb [] 9% 2.1s145.2 Mb [] 9% 2.2s145.2 Mb [] 10% 2.1s145.2 Mb [] 12% 2.0s145.2 Mb [] 13% 1.9s145.2 Mb [] 14% 1.9s145.2 Mb [] 15% 1.8s145.2 Mb [] 17% 1.7s145.2 Mb [] 18% 1.6s145.2 Mb [] 20% 1.5s145.2 Mb [] 22% 1.5s145.2 Mb [] 23% 1.5s145.2 Mb [] 25% 1.4s145.2 Mb [] 26% 1.4s145.2 Mb [] 28% 1.3s145.2 Mb [] 29% 1.3s145.2 Mb [] 30% 1.2s145.2 Mb [] 32% 1.2s145.2 Mb [] 34% 1.2s145.2 Mb [] 35% 1.2s145.2 Mb [] 36% 1.2s145.2 Mb [] 37% 1.1s145.2 Mb [] 38% 1.1s145.2 Mb [] 38% 1.2s145.2 Mb [] 39% 1.1s145.2 Mb [] 41% 1.1s145.2 Mb [] 42% 1.0s145.2 Mb [] 44% 1.0s145.2 Mb [] 47% 0.9s145.2 Mb [] 

In [ ]:
import asyncio
import threading
from concurrent.futures import ThreadPoolExecutor
from faker import Faker
from playwright.async_api import async_playwright
import nest_asyncio

nest_asyncio.apply()
fake = Faker('en_IN')

# Flag to indicate whether the script is running
running = True

async def start(name, user, wait_time, meetingcode, passcode):
    print(f"{name} started!")

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True, args=['--use-fake-device-for-media-stream', '--use-fake-ui-for-media-stream'])
        context = await browser.new_context(permissions=['microphone'])
        page = await context.new_page()
        await page.goto(f'https://zoom.us/wc/join/{meetingcode}', timeout=200000)

        try:
            await page.click('//button[@id="onetrust-accept-btn-handler"]', timeout=5000)
        except Exception as e:
            pass

        try:
            await page.click('//button[@id="wc_agree1"]', timeout=5000)
        except Exception as e:
            pass

        try:
            await page.wait_for_selector('input[type="text"]', timeout=200000)
            await page.fill('input[type="text"]', user)
            await page.fill('input[type="password"]', passcode)
            join_button = await page.wait_for_selector('button.preview-join-button', timeout=200000)
            await join_button.click()
        except Exception as e:
            pass

        try:
            query = '//button[text()="Join Audio by Computer"]'
            await asyncio.sleep(30)
            mic_button_locator = await page.wait_for_selector(query, timeout=200000)
            #await mic_button_locator.wait_for_element_state('stable', timeout=200000)
            #await mic_button_locator.click()
            await mic_button_locator.evaluate_handle('node => node.click()')
            print(f"{name} mic aayenge.")
        except Exception as e:
            print(f"{name} mic nahe aayenge. ", e)

        print(f"{name} sleep for {wait_time} seconds ...")
        while running and wait_time > 0:
            await asyncio.sleep(1)
            wait_time -= 1
        print(f"{name} ended!")

        await browser.close()

async def main():
    global running
    number = int(input("Enter number of Users: "))
    meetingcode = input("Enter meeting code (No Space): ")
    passcode = input("Enter Password (No Space): ")

    sec = 60
    wait_time = sec * 60

    with ThreadPoolExecutor(max_workers=number) as executor:
        loop = asyncio.get_running_loop()
        tasks = []
        for i in range(number):
            try:
                user = fake.name()
            except IndexError:
                break
            task = loop.create_task(start(f'[Thread{i}]', user, wait_time, meetingcode, passcode))
            tasks.append(task)
        try:
            await asyncio.gather(*tasks)
        except KeyboardInterrupt:
            running = False
            # Wait for tasks to complete
            await asyncio.gather(*tasks, return_exceptions=True)

if __name__ == '__main__':

  try:
    asyncio.run(main())
  except KeyboardInterrupt:
    pass


Enter number of Users: 10
Enter meeting code (No Space): 82770760919
Enter Password (No Space): 468111
[Thread0] started!
[Thread1] started!
[Thread2] started!
[Thread3] started!
[Thread4] started!
[Thread5] started!
[Thread6] started!
[Thread7] started!
[Thread8] started!
[Thread9] started!
[Thread6] mic aayenge.
[Thread6] sleep for 3600 seconds ...
[Thread0] mic aayenge.
[Thread0] sleep for 3600 seconds ...
[Thread1] mic aayenge.
[Thread1] sleep for 3600 seconds ...
[Thread5] mic aayenge.
[Thread5] sleep for 3600 seconds ...
[Thread9] mic aayenge.
[Thread9] sleep for 3600 seconds ...
[Thread7] mic aayenge.
[Thread7] sleep for 3600 seconds ...
[Thread8] mic aayenge.
[Thread8] sleep for 3600 seconds ...
[Thread3] mic aayenge.
[Thread3] sleep for 3600 seconds ...
[Thread4] mic aayenge.
[Thread4] sleep for 3600 seconds ...
[Thread2] mic aayenge.
[Thread2] sleep for 3600 seconds ...
